In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, InputLayer
from tensorflow.keras.optimizers import Adam
import os
import numpy as np

2024-10-29 00:00:48.905785: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 00:00:48.923709: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 00:00:48.929346: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 00:00:48.951115: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-29 00:00:49.914047: W tensorflow/compiler/tf2

In [2]:



def parse_file(content):
    x_data=[]
    y_data=[]
    lines = [line.strip() for line in content.split('\n') if line.strip()]
    for line in lines:
        # Remove any whitespace and split by '],['
        arrays = line.replace(' ', '').strip('[]').split('],[')
        
        if len(arrays) == 2:
            try:
                # Parse integers for X
                x_array = [int(x) for x in arrays[0].split(',')]
                # Parse floats for Y
                y_array = [float(x) for x in arrays[1].split(',')]
                
                # Only append if arrays have correct dimensions
                if len(x_array) == 24 and len(y_array) == 12:
                    x_data.append(x_array)
                    y_data.append(y_array)
            except (ValueError, IndexError):
                print(f"Skipping malformed line: {line}")
                continue
    return x_data, y_data

def load_data_from_folder(folder):
    all_x = []
    all_y = []
    data_files = [file for root, dirs, files in os.walk(folder) for file in files if file.endswith('.csv')]
    for data_file in data_files:
        with open(os.path.join(folder,data_file), 'r') as file:
            content = file.read()
            x,y = parse_file(content)
            all_x += x
            all_y += y
    X = np.array(all_x, dtype=np.int32)
    Y = np.array(all_y, dtype= np.float32)
    return X,Y

In [15]:
def create_ffnn_model():
    """
    Creates a Feed-Forward Neural Network model for predicting 
    24 float values from 12 input values
    """
    model = Sequential([
        InputLayer(shape=(12,)),
        # Input layer
        Dense(24, activation='relu'),
        BatchNormalization(),
        
        # Hidden layers
        Dense(48, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        
        Dense(48, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        
        # Output layer
        Dense(24, activation='sigmoid')
    ])
    
    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

def train_model(X_train, y_train, epochs=100, batch_size=32):
    """
    Train the model with the provided data
    
    Parameters:
    X_train: numpy array of shape (n_samples, 12)
    y_train: numpy array of shape (n_samples, 24)
    """
    model = create_ffnn_model()
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True
    )
    
    history = model.fit(
        X_train, 
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )
    
    return model, history

def evaluate_model(model, X_test, y_test):
    """
    Evaluate the model performance
    """
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nTest Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")
    
    # Make some predictions
    predictions = model.predict(X_test[:5])  # First 5 samples
    print("\nSample Predictions vs Actual Values:")
    for i in range(5):
        print(f"\nSample {i+1}:")
        print("Prediction:", predictions[i].round(3))
        print("Actual:", y_test[i])

In [4]:
data_folder = "pcpvectors"
# y,X for maintaining convention from this point onwards
y,X = load_data_from_folder(data_folder) 
print(f"\nLoaded {len(X)} samples from CSV files")
print(f"Input shape: {X.shape}")
print(f"Output shape: {y.shape}")


Loaded 5692 samples from CSV files
Input shape: (5692, 12)
Output shape: (5692, 24)


In [5]:
batch_size = min(32, int(np.sqrt(len(X))))
print(batch_size)
print("\nTraining model...")
model, history = train_model(X,y, epochs=100, batch_size=batch_size)

32

Training model...


I0000 00:00:1730140250.965268   88769 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730140251.018867   88769 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730140251.018924   88769 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730140251.022346   88769 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730140251.022417   88769 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Epoch 1/100


I0000 00:00:1730140254.435977   89958 service.cc:146] XLA service 0x7f9c3c007030 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730140254.436046   89958 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 Ti, Compute Capability 7.5
2024-10-29 00:00:54.485866: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-29 00:00:54.799055: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 48/143 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0930 - loss: 0.8164

I0000 00:00:1730140256.936686   89958 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


143/143 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.1496 - loss: 0.7498 - val_accuracy: 0.1484 - val_loss: 0.4511
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3927 - loss: 0.3761 - val_accuracy: 0.3406 - val_loss: 0.1833
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5241 - loss: 0.1491 - val_accuracy: 0.4390 - val_loss: 0.1342
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5916 - loss: 0.1126 - val_accuracy: 0.4855 - val_loss: 0.1135
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6503 - loss: 0.0955 - val_accuracy: 0.5224 - val_loss: 0.1008
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7088 - loss: 0.0837 - val_accuracy: 0.5531 - val_loss: 0.0933
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7308 - loss: 0.0775 - val_accuracy: 0.6032 - val_loss: 0.0857
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7473 - loss: 0.0712 - val_accuracy: 0.625

In [14]:
print("\nEvaluating model...")
evaluate_model(model, X,y)


Evaluating model...

Test Loss: 0.0355
Test Accuracy: 0.8573
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step

Sample Predictions vs Actual Values:

Sample 1:
Prediction: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]

Sample 2:
Prediction: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]

Sample 3:
Prediction: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]

Sample 4:
Prediction: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]

Sample 5:
Prediction: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]


In [10]:
%%script false --no-raise-error
import matplotlib.pyplot as plt

def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='Training MAE')
    plt.plot(history.history['val_mae'], label='Validation MAE')
    plt.title('Model MAE')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# After training:
plot_training_history(history) 